# Fetch Playlists from Spotify API

In [1]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

In [2]:
# Load data from spotify links
with open('../data/spotify_links.json') as f:
    data = json.load(f)
    links = list(data.items())

# Links consists of tuples of (genre, link)
print('Example:', links[0])

Example: ('italian violin', 'spotify:playlist:6Vf6P26sYyUqljb9e8Jy5b')


## Connect to Spotify API using Spotipy

In [3]:
SPOTIPY_CLIENT_ID = "f96a1f0e67bf41afa9c09b2da29d8c7c"
SPOTIPY_CLIENT_SECRET = "3c726b5c74c643de83771882afc67d2f"

spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID,SPOTIPY_CLIENT_SECRET),
    requests_timeout=5)

In [4]:
# Fetch playlists and dump into json files by batch

playlists = []
failures = []

for i in tqdm(range(len(links)), desc="Fetching..."):
    try:
        playlist = spotify.playlist(links[i][1])
        playlists.append(playlist)
    except:
        failures.append(links[i])

Fetching...: 100%|██████████| 5071/5071 [18:35<00:00,  4.55it/s]


In [11]:
## Format and save playlists to json
newPlaylists = []

for i, pl in enumerate(playlists):
    try:
        newData = dict()
        newData['name'] = pl['name']
        newData['uri'] = links[i][1]
        newData['num_tracks'] = len(pl['tracks']['items'])
        newData['tracks'] =  []
        for item in pl['tracks']['items']:
            newTrack = dict()
            newTrack['artists'] = item['track']['artists']
            newTrack['name'] = item['track']['name']
            newTrack['uri'] = item['track']['uri']
            newData['tracks'].append(newTrack)
        newPlaylists.append(newData)      
        if (i + 1) % 200 == 0:
            begin = (i + 1) - 200
            with open('../data/genre_playlists/playlists.{}-{}.json'.format(begin, i), 'w') as outfile:
                json.dump(newPlaylists, outfile, indent=4)
            newPlaylists = []
        elif i == len(playlists) - 1:
            begin = i - (i + 1) % 200
            with open('../data/genre_playlists/playlists.{}-{}.json'.format(begin, i), 'w') as outfile:
                json.dump(newPlaylists, outfile, indent=4)
            newPlaylists = []
    except:
        pass